<a href="https://colab.research.google.com/github/kurniawano/swift-notes/blob/master/Protocol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Protocol

A protocol defines a blueprint of methods, properties, and other requirements that suit a particular task or piece of functionality. The protocol can then be adopted by a class, structure, or enumeration to provide an actual implementation of those requirements. Any type that satisfies the requirements of a protocol is said to conform to that protocol.

## Syntax

### Property Requirement

In [0]:
protocol StateMachine{
  var startState: Int {get set} // property requirements
}

The above protocol specifies that any type conforming to StateMachine protocol must have a `startState` property. 

In [0]:
class CM: StateMachine{ // indicate that SM conform to StateMachine Protocol
  var startState: Int = 0
}

Class `SM` conforms to `StateMachine` protocol and so must implement the properties `startState``.

In [0]:
let cm = CM()
print(cm.startState)

0


### Method Requirement

In [0]:
protocol StateMachine: AnyObject{ // AnyObject ensures that only class-type conforms to StateMachine, not struct or enum, not necessray here
  var startState: Int? {get set} 
  
  // Method requirement
  func getNextValues(state: Int?, inp: Int) -> (Int?, Int?)
}

Creating a base class `SM` for all State Machine class.

In [0]:
class SM{ 
  var startState: Int?
  var state: Int?

  func start(){
    state = startState
  }

  func step(inp: Int) -> Int?{
    var nextState: Int?
    var output: Int?
    (nextState, output) = getNextValues(state: state, inp: inp)
    state = nextState
    return output
  }

  func transduce(listInput: [Int]) -> [Int?]{
    var output = [Int?]()
    start()
    for inp in listInput{
      output.append(step(inp: inp))
    }
    return output
  }

  func getNextValues(state: Int?, inp: Int) -> (Int?, Int?){
    preconditionFailure("This method must be overridden")
    //return (0,0)
  }
}

Any other state machine class inherits from SM and must conform to StateMachine protocol.

In [0]:
class CM: SM, StateMachine{ // CM inherits from SM and conforms to StateMachine Protocol
  override init(){
    super.init()
    startState = 0
  }
  override func getNextValues(state: Int?, inp: Int) -> (Int?, Int?){
    var nextState: Int 
    var output: Int 
    if let curState = state{
      if curState == 0{
        if inp == 100{
          nextState = 0
          output = 1
        } else if inp == 50{
          nextState = 1
          output = 0
        } else{
          nextState = 0
          output = 0
        }
      } else{
        if inp == 50 {
          nextState = 0
          output = 1
        } else if inp == 100{
          nextState = 0
          output = 1
        } else{
          nextState = 1
          output = 0
        }
      }
      return (nextState, output)
    } else{
      return (nil, nil)
    }
  }
}

In [0]:
let cm = CM()
print(cm.transduce(listInput: [50, 50, 100, 10]))

[Optional(0), Optional(1), Optional(1), Optional(0)]


## Generics and Protocol

When defining a protocol, it’s sometimes useful to declare one or more associated types as part of the protocol’s definition. An associated type gives a placeholder name to a type that is used as part of the protocol. The actual type to use for that associated type isn’t specified until the protocol is adopted. Associated types are specified with the associatedtype keyword.

In [0]:
protocol StateMachine: AnyObject{ // AnyObject ensures that only class-type conforms to StateMachine, not struct or enum

  // define associated type here
  associatedtype State
  associatedtype Input
  associatedtype Output

  // we use State associated type for startState
  var startState: State {get set} 
  
  func getNextValues(state: State, inp: Input) -> (State, Output?)
}

In [0]:
class SM<State, Input, Output>{ 

  final var startState: State
  final var state: State? // final to prevent override

  init(startState: State){
    self.startState = startState
  }

  final func start(){
    state = startState
  }

  final func step(inp: Input?) -> Output?{
    var nextState: State?
    var output: Output?
    
    precondition(inp != nil, "Input cannot be nil.")
    let curInput = inp!
    
    if let curState = state{
      (nextState, output) = getNextValues(state: curState, inp: curInput)
    } else{
      preconditionFailure("You must run start() to initialize the state before step(), or use transduce().")
    }
    state = nextState
    return output
  }

  final func transduce(listInput: [Input]) -> [Output?]{
    var output = [Output?]()
    start()
    for inp in listInput{
      output.append(step(inp: inp))
    }
    return output
  }

  func getNextValues(state: State, inp: Input) -> (State, Output?){
    preconditionFailure("This method must be overridden.")
  }
}

In [0]:
class CM: SM<Int,Int,(Int, String, Int)>, StateMachine{ // CM inherits from SM and conforms to StateMachine Protocol

  typealias State = Int 
  typealias Input = Int 
  typealias Output = (Int, String, Int) // (coin inside, output, change)

  init(){
    super.init(startState: 0)
  }
  override func getNextValues(state: State, inp: Input) -> (State, Output?){
    var nextState: State 
    var output: Output 
    if state == 0{
        if inp == 100{
          nextState = 0
          output = (0, "Coke", 0)
        } else if inp == 50{
          nextState = 1
          output = (50, "", 0)
        } else{
          nextState = 0
          output = (inp, "", inp)
        }
      } else{
        if inp == 50 {
          nextState = 0
          output = (0, "Coke", 0)
        } else if inp == 100{
          nextState = 0
          output = (0, "Coke", 50)
        } else{
          nextState = 1
          output = (50, "", inp)
        }
      }
      return (nextState, output)
  }
}

In [29]:
let cm = CM()
cm.start()
print(cm.step(inp:50))
print(cm.transduce(listInput: [50, 50, 100, 10]))

Optional((50, "", 0))
[Optional((50, "", 0)), Optional((0, "Coke", 0)), Optional((0, "Coke", 0)), Optional((10, "", 10))]
